In [1]:
import pandas as pd
import pymysql
import os

def connect_db():
    try:
        #建立connect路径
        conn = pymysql.connect(host = 'sql.b51.vhostgo.com', db = 'passvisor', user = 'passvisor', password = 'Xudamin123', charset = 'utf8')
        #建立cursor
        cur = conn.cursor()
        return conn, cur    #返回路径、cursor，供下文使用
    except:
        return False       #如上述代码请求不成功，返回False字样

def read_index():
    #读入
    f = open('index', encoding = 'utf-8')
    lines = f.readlines()
    f.close()
    print('已读入原始文件')
    return lines

    #改写
    #改写7天预测数值
def modify_index(lines):
    conn, cur = connect_db()
    sql = "select * from predicted_value where date = (select max(date) from predicted_value)"
    cur.execute(sql)
    p_values = cur.fetchall()
    newest_date = p_values[0][1]
    
    _temp = list(lines[74])
    _temp.insert(18,str(p_values[0][4]))
    lines[74] = ''.join(_temp)

    _temp = list(lines[75])
    _temp.insert(18,str(p_values[0][5]))
    lines[75] = ''.join(_temp)

    _temp = list(lines[76])
    _temp.insert(18,str(p_values[0][6]))
    lines[76]= ''.join(_temp)

    _temp = list(lines[77])
    _temp.insert(18,str(p_values[0][7]))
    lines[77] = ''.join(_temp)

    _temp = list(lines[78])
    _temp.insert(18,str(p_values[0][8]))
    lines[78] = ''.join(_temp)

    _temp = list(lines[79])
    _temp.insert(18,str(p_values[0][9]))
    lines[79] = ''.join(_temp)

    _temp = list(lines[80])
    _temp.insert(18,str(p_values[0][10]))
    lines[80] = ''.join(_temp)
    
    #改写预测计算日期的显示
    _temp = list(lines[94])
    _temp.insert(37,str(newest_date))
    lines[94] = ''.join(_temp)
    
    print('文件改写完成')
    return lines

    #保存
def write_index(lines):
    import os 
    if os.path.exists('index.html'):
        os.remove('index.html')

    f = open('index.html', 'a+', encoding = 'utf-8')
    for i in lines: 
        f.write(i)
    f.close()
    print('成功保存文件。完成！')

def read_js():
    f = open('old_index', encoding = 'utf-8')
    lines = f.readlines()
    f.close()
    return lines

def modify_js(lines):
    conn, cur = connect_db()

    #相关系数改写
    sql = "select * from copredict_corr where date = (select max(date) from copredict_corr)"
    cur.execute(sql)
    c_values = cur.fetchall()

    _temp = list(lines[3])
    _temp.insert(25,str(c_values[0][2]))
    _temp.insert(28,str(c_values[0][3]))
    lines[3] = ''.join(_temp)



    #价格历史记录改写
    sql = 'select year(date), month(date), avg(price) from copredict_source_pork group by year(date), month(date)'
    cur.execute(sql)
    avg_month = cur.fetchall()

    #横坐标赋值
    month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    _temp = list(lines[24])
    k = avg_month[len(avg_month)-1][1]

    for i in range(12):
        if k <12:
            k+= 1
        else:
            k = k + 1 -12
        _temp.insert(11+5*i, month_list[k-1])

    lines[24] = ''.join(_temp)


    #纵坐标赋值-第1行
    _temp = list(lines[26])

    for i in range(6):
        _temp.insert(13+i*3, str(round(avg_month[len(avg_month)-12+i][2],1)))

    lines[26] = ''.join(_temp)


    #纵坐标赋值-第2行
    _temp = list(lines[27])

    for i in range(6):
        _temp.insert(1+i*3, str(round(avg_month[len(avg_month)-12+i+6][2],1)))

    lines[27] = ''.join(_temp)
    return lines
    
def write_js(lines):
    if os.path.exists('old_index.js'):
        os.remove('old_index.js')

    f = open('old_index.js', 'a+', encoding = 'utf-8')
    for i in lines: 
        f.write(i)
    f.close()
    
def main():
    lines = read_index()
    lines = modify_index(lines)
    write_index(lines)
    lines = read_js()
    lines = modify_js(lines)
    write_js(lines)

if __name__ == '__main__':
    main()

已读入原始文件
文件改写完成
成功保存文件。完成！


In [2]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-


from ftplib import FTP
import os
import sys
import time
import socket


class MyFTP:

    def __init__(self, host, port=21):
        """ 初始化 FTP 客户端
        参数:
                 host:ip地址

                 port:端口号
        """
        # print("__init__()---> host = %s ,port = %s" % (host, port))

        self.host = host
        self.port = port
        self.ftp = FTP()
        # 重新设置下编码方式
        self.ftp.encoding = 'gbk'
        self.log_file = open("log.txt", "a")
        self.file_list = []

    def login(self, username, password):
        """ 初始化 FTP 客户端
            参数:
                  username: 用户名

                 password: 密码
            """
        try:
            timeout = 60
            socket.setdefaulttimeout(timeout)
            # 0主动模式 1 #被动模式
            self.ftp.set_pasv(False)
            # 打开调试级别2，显示详细信息
            # self.ftp.set_debuglevel(2)

            self.debug_print('开始尝试连接到 %s' % self.host)
            self.ftp.connect(self.host, self.port)
            self.debug_print('成功连接到 %s' % self.host)

            self.debug_print('开始尝试登录到 %s' % self.host)
            self.ftp.login(username, password)
            self.debug_print('成功登录到 %s' % self.host)

            self.debug_print(self.ftp.welcome)
        except Exception as err:
            self.deal_error("FTP 连接或登录失败 ，错误描述为：%s" % err)
            pass

    def is_same_size(self, local_file, remote_file):
        """判断远程文件和本地文件大小是否一致

           参数:
             local_file: 本地文件

             remote_file: 远程文件
        """
        try:
            remote_file_size = self.ftp.size(remote_file)
        except Exception as err:
            # self.debug_print("is_same_size() 错误描述为：%s" % err)
            remote_file_size = -1

        try:
            local_file_size = os.path.getsize(local_file)
        except Exception as err:
            # self.debug_print("is_same_size() 错误描述为：%s" % err)
            local_file_size = -1

        self.debug_print('local_file_size:%d  , remote_file_size:%d' % (local_file_size, remote_file_size))
        if remote_file_size == local_file_size:
            return 1
        else:
            return 0

    def download_file(self, local_file, remote_file):
        """从ftp下载文件
            参数:
                local_file: 本地文件

                remote_file: 远程文件
        """
        self.debug_print("download_file()---> local_path = %s ,remote_path = %s" % (local_file, remote_file))

        if self.is_same_size(local_file, remote_file):
            self.debug_print('%s 文件大小相同，无需下载' % local_file)
            return
        else:
            try:
                self.debug_print('>>>>>>>>>>>>下载文件 %s ... ...' % local_file)
                buf_size = 1024
                file_handler = open(local_file, 'wb')
                self.ftp.retrbinary('RETR %s' % remote_file, file_handler.write, buf_size)
                file_handler.close()
            except Exception as err:
                self.debug_print('下载文件出错，出现异常：%s ' % err)
                return

    def download_file_tree(self, local_path, remote_path):
        """从远程目录下载多个文件到本地目录
                       参数:
                         local_path: 本地路径

                         remote_path: 远程路径
                """
        print("download_file_tree()--->  local_path = %s ,remote_path = %s" % (local_path, remote_path))
        try:
            self.ftp.cwd(remote_path)
        except Exception as err:
            self.debug_print('远程目录%s不存在，继续...' % remote_path + " ,具体错误描述为：%s" % err)
            return

        if not os.path.isdir(local_path):
            self.debug_print('本地目录%s不存在，先创建本地目录' % local_path)
            os.makedirs(local_path)

        self.debug_print('切换至目录: %s' % self.ftp.pwd())

        self.file_list = []
        # 方法回调
        self.ftp.dir(self.get_file_list)

        remote_names = self.file_list
        self.debug_print('远程目录 列表: %s' % remote_names)
        for item in remote_names:
            file_type = item[0]
            file_name = item[1]
            local = os.path.join(local_path, file_name)
            if file_type == 'd':
                print("download_file_tree()---> 下载目录： %s" % file_name)
                self.download_file_tree(local, file_name)
            elif file_type == '-':
                print("download_file()---> 下载文件： %s" % file_name)
                self.download_file(local, file_name)
            self.ftp.cwd("..")
            self.debug_print('返回上层目录 %s' % self.ftp.pwd())
        return True

    def upload_file(self, local_file, remote_file):
        """从本地上传文件到ftp

           参数:
             local_path: 本地文件

             remote_path: 远程文件
        """
        if not os.path.isfile(local_file):
            self.debug_print('%s 不存在' % local_file)
            return

        #if self.is_same_size(local_file, remote_file):
            #self.debug_print('跳过相等的文件: %s' % local_file)
            #return

        buf_size = 1024
        file_handler = open(local_file, 'rb')
        self.ftp.storbinary('STOR %s' % remote_file, file_handler, buf_size)
        file_handler.close()
        self.debug_print('上传: %s' % local_file + "成功!")

    def upload_file_tree(self, local_path, remote_path):
        """从本地上传目录下多个文件到ftp
           参数:

             local_path: 本地路径

             remote_path: 远程路径
        """
        if not os.path.isdir(local_path):
            self.debug_print('本地目录 %s 不存在' % local_path)
            return
        """
        创建服务器目录
        """
        try:
            self.ftp.cwd(remote_path)  # 切换工作路径
        except Exception as e:
            base_dir, part_path = self.ftp.pwd(), remote_path.split('/')
            for p in part_path[1:-1]:
                base_dir = base_dir + p + '/'  # 拼接子目录
                try:
                    self.ftp.cwd(base_dir)  # 切换到子目录, 不存在则异常
                except Exception as e:
                    print('INFO:', e)
                    self.ftp.mkd(base_dir)  # 不存在创建当前子目录
        #self.ftp.cwd(remote_path)
        self.debug_print('切换至远程目录: %s' % self.ftp.pwd())

        local_name_list = os.listdir(local_path)
        self.debug_print('本地目录list: %s' % local_name_list)
        #self.debug_print('判断是否有服务器目录: %s' % os.path.isdir())

        for local_name in local_name_list:
            src = os.path.join(local_path, local_name)
            print("src路径=========="+src)
            if os.path.isdir(src):
                try:
                    self.ftp.mkd(local_name)
                except Exception as err:
                    self.debug_print("目录已存在 %s ,具体错误描述为：%s" % (local_name, err))
                self.debug_print("upload_file_tree()---> 上传目录： %s" % local_name)
                self.debug_print("upload_file_tree()---> 上传src目录： %s" % src)
                self.upload_file_tree(src, local_name)
            else:
                self.debug_print("upload_file_tree()---> 上传文件： %s" % local_name)
                self.upload_file(src, local_name)
        self.ftp.cwd("..")

    def close(self):
        """ 退出ftp
        """
        self.debug_print("close()---> FTP退出")
        self.ftp.quit()
        self.log_file.close()

    def debug_print(self, s):
        """ 打印日志
        """
        self.write_log(s)

    def deal_error(self, e):
        """ 处理错误异常
            参数：
                e：异常
        """
        log_str = '发生错误: %s' % e
        self.write_log(log_str)
        sys.exit()

    def write_log(self, log_str):
        """ 记录日志
            参数：
                log_str：日志
        """
        time_now = time.localtime()
        date_now = time.strftime('%Y-%m-%d', time_now)
        format_log_str = "%s ---> %s \n " % (date_now, log_str)
        print(format_log_str)
        self.log_file.write(format_log_str)

    def get_file_list(self, line):
        """ 获取文件列表
            参数：
                line：
        """
        file_arr = self.get_file_name(line)
        # 去除  . 和  ..
        if file_arr[1] not in ['.', '..']:
            self.file_list.append(file_arr)

    def get_file_name(self, line):
        """ 获取文件名
            参数：
                line：
        """
        pos = line.rfind(':')
        while (line[pos] != ' '):
            pos += 1
        while (line[pos] == ' '):
            pos += 1
        file_arr = [line[0], line[pos:]]
        return file_arr


if __name__ == "__main__":
    my_ftp = MyFTP("passvisor.gotoftp5.com")
    #my_ftp.set_pasv(False)
    my_ftp.login("passvisor", "Xudamin123")

    # 下载单个文件
    #my_ftp.download_file("/home/BG_2019_05_22_16_04_54_Camera6-0.mp4", "/BG_2019_05_22_16_04_54_Camera6-0.mp4") #FTP服务器目录   本地目录

    # 下载目录
    # my_ftp.download_file_tree("G:/ftp_test/", "App/AutoUpload/ouyangpeng/I12/")

    # 上传单个文件
    my_ftp.upload_file("index.html", "wwwroot/mpl/index.html")
    my_ftp.upload_file("old_index.js", "wwwroot/mpl/assets/js/old_index.js")

    # 上传目录
    #my_ftp.upload_file_tree("/home/zoukaicai/java8", "/123/5/")

    my_ftp.close()


2020-02-26 ---> 开始尝试连接到 passvisor.gotoftp5.com 
 
2020-02-26 ---> 成功连接到 passvisor.gotoftp5.com 
 
2020-02-26 ---> 开始尝试登录到 passvisor.gotoftp5.com 
 
2020-02-26 ---> 成功登录到 passvisor.gotoftp5.com 
 
2020-02-26 ---> 220-Microsoft FTP Service
220 欢迎你连接本站FTP! 
 
2020-02-26 ---> 上传: index.html成功! 
 
2020-02-26 ---> 上传: old_index.js成功! 
 
2020-02-26 ---> close()---> FTP退出 
 
